# Probe IL2/15/21 Essentiality in DepMap/CCLE

In [ ]:
import pandas as pd
import plotly.express as px

## 1. Load Data and Set Cytokines

In [ ]:
dm_sample_info = pd.read_csv('../../data/dm_sample_info.csv', index_col=0)
dm_gene_expression = pd.read_csv('../../data/CCLE_expression.csv', index_col=0)
dependency = pd.read_csv('../../data/CRISPR_gene_dependency.csv', index_col=0)
CYTOKINES = ['IL2', 'IL15', 'IL21']

## 2. Reformat Data to Enable Plotting

In [ ]:
lineage = dm_sample_info['lineage']
lineage

In [ ]:
dm_gene_expression.columns = [i.split(' ')[0] for i in dm_gene_expression.columns]
dm_gene_expression

In [ ]:
# Plotly Data Formating Fucntion
def data_to_plotly(data, cytokines=CYTOKINES):
    values = []
    labels = []
    index = []

    for cytokine in cytokines:
        values += [*data[cytokine].values]
        labels += [cytokine]*data[cytokine].shape[0]
        index += [*data.index]

    return (values, labels, index)

## 3. Make One Histogram with All Cytokines

In [ ]:
# Format the Data
values, labels, _ = data_to_plotly(dm_gene_expression)
big_px_df = pd.DataFrame([values, labels], index=['gene_expression', 'gene'], dtype=float)
big_px_df

In [ ]:
# Make the Graph
px.histogram(
    data_frame=big_px_df.T,
    x='gene_expression',
    color='gene',
    barmode='overlay',
    title="Average Cytokine Gene Expression")

#### Group by Mean Gene Expression `p`

In [ ]:
dm_ge_x_lineage = dm_gene_expression.groupby(lineage).mean()

In [ ]:
# Format the Data
values, labels, index = data_to_plotly(dm_ge_x_lineage)
big_px_df = pd.DataFrame([values, labels, index], index=['average_gene_expression', 'gene', 'tissue_lineage'])
big_px_df

In [ ]:
# Make the Graph
px.bar(
    data_frame=big_px_df.T,
    x='tissue_lineage',
    y='average_gene_expression',
    color='gene',
    barmode='overlay',
    title='Average Cytokine Gene Expression by Tissue Lineage',
    height=1000)

## 4. Graph Cytokine Gene Dependency Score

In [ ]:
# Format the Data
values, labels, _ = data_to_plotly(dependency)
big_px_df = pd.DataFrame([values, labels], index=['gene_dependency_score', 'gene'])

In [ ]:
# Make the Graph
px.histogram(
    data_frame=big_px_df.T,
    x='gene_dependency_score',
    color='gene',
    barmode='overlay',
    title='Cytokine Gene Dependency Score')

#### Group by Mean Gene Expression `p`

In [ ]:
dm_dep_x_lineage = dependency.groupby(lineage).mean()

In [ ]:
# Format the Data
values, labels, index = data_to_plotly(dm_dep_x_lineage)
big_px_df = pd.DataFrame([values, labels, index], index=['average_gene_dependency', 'gene', 'tissue_lineage'])
big_px_df

In [ ]:
# Make the Graph
px.bar(
    data_frame=big_px_df.T,
    x='tissue_lineage',
    y='average_gene_dependency', 
    color='gene',
    barmode='overlay',
    title='Average Cytokine Gene  Dependency Score by Tissue Lineage',
    height=1000)

## 5. Alternatively, Make Different Graphs for Each Cytokine

Start with Expression 

In [ ]:
# Transpose the DataFrame
dm_gene_expression = dm_gene_expression.T

In [ ]:
for cytokine in CYTOKINES:
    # Start with all Cytokines
    fig=px.histogram(
        data_frame=dm_gene_expression.loc[cytokine].to_frame(),
        x=cytokine,
        range_x=[0,8],
        nbins=200,
        title=f'{cytokine}: Gene Expression',
        labels={cytokine: 'Gene Expression Value'})
    fig.show()
    
    # Then do the group by
    fig=px.bar(
        dm_gene_expression.T.groupby(lineage).mean().loc[:, cytokine],
        title=f'Average {cytokine} Expression by Tissue Lineage')
    fig.show()

Now do the same for Dependency

In [ ]:
# Transpose the DataFrame
dependency=dependency.T

In [ ]:
for cytokine in CYTOKINES:
    # Start with all Cytokines
    fig=px.histogram(
        x=dependency.loc[cytokine].values,
        range_x=[0,1],
        nbins=200,
        title=f'{cytokine}: Dependency')
    fig.show()

    # Then do the group by
    fig=px.bar(
        dependency.T.groupby(lineage).mean().loc[:, cytokine],
        title=f'Average {cytokine} Dependency by Tissue Lineage')
    fig.show()